Installing required libraries

In [1]:
#After executing the cell,please RESTART the kernel and run all the cells.
!pip install --user "ibm-watsonx-ai==1.1.2"
!pip install --user "langchain==0.2.11"
!pip install --user "langchain-ibm==0.1.11"
!pip install --user "langchain-community==0.2.10"
!pip install --user "sentence-transformers==3.0.1"

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 966.6/966.6 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 42.4 MB/s eta 0:00:0000:01
  Created wheel for ibm-cos-sdk: filename=ibm_cos_sdk-2.13.6-py3-none-any.whl size=77322 sha256=796ba09651df91786bf06ea6c447904c7c6223d9dd48d90bb0b5038c016d9b58
  Stored in directory: /home/codespace/.cache/pip/wheels/17/c5/52/64bd703cf3cc4a83ba0e39d9b3eedc3a15afced69bbe6

Loading file

In [3]:
#!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/i5V3ACEyz6hnYpVq6MTSvg/state-of-the-union.txt"
from langchain_community.document_loaders import TextLoader
loader = TextLoader("state-of-the-union.txt")
data = loader.load()
data[0]

Document(metadata={'source': 'state-of-the-union.txt'}, page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determina

Split data

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
)

chunks = text_splitter.split_text(data[0].page_content)
len(chunks)

570

In [6]:
chunks[0]

'Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and'

Embedding Model (WatBERT : slate-125m-english-rtrvr) : model is a standard sentence transformers model based on bi-encoders.
<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/NDCHhZfcC96jggb2hMdJhg/fm-slate-125m-english-rtrvr-cosine.jpg" width="50%">


In [8]:
#Imports
from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames
from langchain_ibm import WatsonxEmbeddings

Setting Watson KEY as Environment variable

In [ ]:
import os

os.environ['WATSONX_APIKEY'] = 'Replace IBM Key'

# Your code here

In [15]:
embed_params = {
    EmbedTextParamsMetaNames.TRUNCATE_INPUT_TOKENS: 3,
    EmbedTextParamsMetaNames.RETURN_OPTIONS: {"input_text": True},
}

watsonx_embedding = WatsonxEmbeddings(
    model_id="ibm/slate-125m-english-rtrvr",
    url="https://eu-de.ml.cloud.ibm.com",
    project_id="7ce63394-6c4b-4e32-8909-456031209bab",
    params=embed_params,
)

Embedd

In [16]:
query = "How are you?"
query_result = watsonx_embedding.embed_query(query)

len(query_result)

768

Document embedding

In [ ]:
doc_result = watsonx_embedding.embed_documents(chunks)
len(doc_result) # chunks

570

In [ ]:
len(doc_result[0]) # embedding per chunk same as earlier query

768

Embedding using hugging face

In [20]:
from langchain_community.embeddings import HuggingFaceEmbeddings
model_name = "sentence-transformers/all-mpnet-base-v2"
huggingface_embedding = HuggingFaceEmbeddings(model_name=model_name)

/tmp/ipykernel_7760/1021890160.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  huggingface_embedding = HuggingFaceEmbeddings(model_name=model_name)
/home/codespace/.local/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [21]:
query = "How are you?"
query_result = huggingface_embedding.embed_query(query)
len(query_result)


768

In [23]:
doc_result = huggingface_embedding.embed_documents(chunks)
doc_result[0][:5]

[0.040284302085638046,
 0.030196329578757286,
 -0.00639386149123311,
 0.06266564875841141,
 -0.007770895957946777]

In [ ]:
len(doc_result) #Same number of chunks

570

In [ ]:
len(doc_result[0]) #Same as query

768

Another Watson model

In [ ]:
from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames
from langchain_ibm import WatsonxEmbeddings

embed_params = {
    EmbedTextParamsMetaNames.TRUNCATE_INPUT_TOKENS: 3,
    EmbedTextParamsMetaNames.RETURN_OPTIONS: {"input_text": True},
}

watsonx_embedding = WatsonxEmbeddings(
    model_id="ibm/slate-30m-english-rtrvr",
    url="https://eu-de.ml.cloud.ibm.com",
    project_id="7ce63394-6c4b-4e32-8909-456031209bab",
    params=embed_params,
)

doc_result = watsonx_embedding.embed_documents(chunks)

len(doc_result[0]) #Less embedding

384